<a href="https://colab.research.google.com/github/Joeri5/Web-Image-Scraper/blob/main/download_images.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get update
!apt-get install firefox -y
!apt install -y wget
!apt install -y unzip
!pip install selenium

# Download geckodriver
!wget https://github.com/mozilla/geckodriver/releases/download/v0.30.0/geckodriver-v0.30.0-linux64.tar.gz
!tar -xvzf geckodriver-v0.30.0-linux64.tar.gz

# Move the geckodriver binary to /usr/bin
!mv geckodriver /usr/bin/

In [ ]:
import requests
import os
from urllib.parse import urlparse, urljoin
from selenium import webdriver
from selenium.webdriver.common.by import By
import time

def download_images(url):
    # Set up the Firefox driver and visit the URL
    options = webdriver.FirefoxOptions()
    options.add_argument("--headless")  # Use headless mode to avoid opening a browser window
    driver = webdriver.Firefox(options=options)
    driver.get(url)

    # Check for cookie popup and reject if it exists
    try:
        cookie_popup = driver.find_element(By.XPATH, "//div[contains(@class, 'cookie')]")
        reject_button = cookie_popup.find_element(By.XPATH, "//button[contains(text(), 'Reject') or contains(text(), 'Decline')]")
        reject_button.click()
        time.sleep(2)  # Wait for the page to reload after rejecting cookies
    except:
        pass

    # Wait for all content to load
    time.sleep(5)  # You may need to adjust the wait time depending on the website

    # Scroll to the bottom of the page
    scroll_height = driver.execute_script("return document.body.scrollHeight;")
    current_position = 0
    while current_position < scroll_height:
        current_position += 1000  # Decrease the scroll speed by changing this value
        driver.execute_script(f"window.scrollTo(0, {current_position});")
        time.sleep(1)  # You may need to adjust the wait time depending on the website

    # Create a directory to store the images
    dir = 'images'
    if not os.path.exists(dir):
        os.makedirs(dir)

    # Find all image tags and download the images
    for img in driver.find_elements(By.TAG_NAME, "img"):
        img_url = img.get_attribute("src")
        if img_url and (img_url.endswith(".png") or img_url.endswith(".jpg") or img_url.endswith(".jpeg") or img_url.endswith(".svg") or img_url.endswith(".png")):
            img_url = urljoin(url, img_url)
            print(f"Downloading {img_url}...")
            img_response = requests.get(img_url)
            filename = os.path.join(dir, os.path.basename(img_url))
            with open(filename, "wb") as f:
                f.write(img_response.content)
                print(f"Downloaded {filename}")

    # Quit the driver
    driver.quit()
    
# Example usage:
url = "https://www.apple.com/nl/iphone-14/"
download_images(url)


In [ ]:
import os
import zipfile
from google.colab import files

# Create a new 'downloads' directory
downloads_dir = '/content/downloads'
if not os.path.exists(downloads_dir):
    os.makedirs(downloads_dir)

# Replace 'folder_name' with the name of the folder you want to zip
folder_name = 'images'

# Zip the folder
zip_filename = f'{folder_name}.zip'
zip_filepath = f'{downloads_dir}/{zip_filename}'
with zipfile.ZipFile(zip_filepath, 'w', zipfile.ZIP_DEFLATED) as zip_file:
    for root, dirs, files in os.walk(folder_name):
        for file in files:
            zip_file.write(os.path.join(root, file))